In [1]:
# dependencies
!pip install psutil torch pandas

import torch
import torchvision.models as models
import time
import psutil
import os
import pandas as pd

# Load pre-trained SqueezeNet model
model = models.squeezenet1_1(pretrained=True)
model.eval()

# Use Apple M1/M2 GPU if available
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

# Dummy input tensor
dummy_input = torch.randn(1, 3, 224, 224).to(device)

# Run benchmark
runs = 100
inference_times = []

for i in range(runs):
    start = time.time()
    with torch.no_grad():
        _ = model(dummy_input)
    end = time.time()
    inference_times.append(end - start)

# Summary stats
avg_time = sum(inference_times) / runs
std_dev = (sum((x - avg_time) ** 2 for x in inference_times) / runs) ** 0.5
ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

print(f"Device: {device}")
print(f"Average Inference Time: {avg_time:.4f} s")
print(f"Standard Deviation: {std_dev:.4f} s")
print(f"RAM Usage: {ram_usage:.2f} MB")

# Save results to DataFrame for later export
df = pd.DataFrame({
    'Run': list(range(1, runs + 1)),
    'Inference Time (s)': inference_times
})
df.head()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 8.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


/Users/belix/.pyenv/versions/3.12.10/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/belix/.pyenv/versions/3.12.10/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth" to /Users/belix/.cache/torch/hub/checkpoints/squeezenet1_1-b8a52dc0.pth
100.0%


Device: cpu
Average Inference Time: 0.0455 s
Standard Deviation: 0.0218 s
RAM Usage: 299.79 MB


,Run,Inference Time (s)
0,1,0.172295
1,2,0.085132
2,3,0.041712
3,4,0.065861
4,5,0.047541


In [2]:
df.to_csv("squeezenetbenchmark.csv", index=False)